In [1]:
! pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from functools import lru_cache
from Levenshtein import distance as levenshtein_distance
from collections import defaultdict
import numpy as np

# Прочтение таблиц

In [4]:
df = pd.read_excel('/content/FEV.xlsx')
df.head()

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,5160,0.003694,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1
1,1,2317,0.001659,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5
2,2,2154,0.001542,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1
3,3,2073,0.001484,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4
4,4,1583,0.001133,TGTGCCAGCAGCCCGGGAGATCAGCCCCAGCATTTT,CASSPGDQPQHF,TRBV7-9,TRBJ1-5


In [6]:
database = pd.read_excel('/content/vdjdb_slim.xlsx')

In [7]:
 database_exp = database.rename(columns={'cdr3': 'CDR3.amino.acid.sequence'})

In [8]:
data_claster = df[:10000]
data_claster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene
0,0,5160,0.003694,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1
1,1,2317,0.001659,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5
2,2,2154,0.001542,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1
3,3,2073,0.001484,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4
4,4,1583,0.001133,TGTGCCAGCAGCCCGGGAGATCAGCCCCAGCATTTT,CASSPGDQPQHF,TRBV7-9,TRBJ1-5
...,...,...,...,...,...,...,...
9995,10956,7,0.000005,TGTGCCACCAGTGACCTGACAGAAGAGACCCAGTACTTC,CATSDLTEETQYF,TRBV24-1,TRBJ2-5
9996,9786,7,0.000005,TGTGCCACCAGTGACGGACAGACCCACACAGATACGCAGTATTTT,CATSDGQTHTDTQYF,TRBV24-1,TRBJ2-3
9997,10388,7,0.000005,TGTGCCACCAGTGACGGACAGGGTTACAATGAGCAGTTCTTC,CATSDGQGYNEQFF,TRBV24-1,TRBJ2-1
9998,10389,7,0.000005,TGTGCCACCAGTGAGCTGGAACGGGCCGGGGAGCTGTTTTTT,CATSELERAGELFF,TRBV24-1,TRBJ2-2


# Функция кластеризации

In [9]:
# Кэшированная функция для вычисления расстояния Левенштейна
@lru_cache(maxsize=None)
def cached_levenshtein_distance(seq1, seq2):
    return levenshtein_distance(seq1, seq2)

# Функция для кластеризации последовательностей
def cluster_sequences(sequences, max_distance=1):
    clusters = []
    for seq in sequences:
        added_to_cluster = False
        for cluster in clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for existing_seq in cluster):
                cluster.append(seq)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([seq])
    return clusters

# Функция для объединения кластеров
def merge_clusters(clusters, max_distance=1):
    merged_clusters = []
    for cluster in clusters:
        added_to_cluster = False
        for merged_cluster in merged_clusters:
            if any(cached_levenshtein_distance(seq, existing_seq) <= max_distance for seq in cluster for existing_seq in merged_cluster):
                merged_cluster.extend(cluster)
                added_to_cluster = True
                break
        if not added_to_cluster:
            merged_clusters.append(cluster)
    return merged_clusters

# Пример последовательностей аминокислот
sequences = data_claster['CDR3.amino.acid.sequence']

# Разделение данных на части
chunk_size = len(sequences) // 100
chunks = [sequences[i:i + chunk_size] for i in range(0, len(sequences), chunk_size)]

# Кластеризация каждой части
all_clusters = []
for chunk in chunks:
    all_clusters.extend(cluster_sequences(chunk, max_distance=1))

# Объединение кластеров
final_clusters = merge_clusters(all_clusters, max_distance=1)

# Вывод результатов
# Создание DataFrame
cluster_data = []
for i, cluster in enumerate(final_clusters):
    for seq in cluster:
        cluster_data.append({'Cluster': i + 1, 'Sequence': seq})

results_df = pd.DataFrame(cluster_data)

In [9]:
# Создание нового DataFrame с данными из data_claster и номером кластера
cluster_assignments = {}
for i, cluster in enumerate(final_clusters):
  for seq in cluster:
    cluster_assignments[seq] = i + 1

results_df = data_claster.copy()
results_df['Cluster'] = results_df['CDR3.amino.acid.sequence'].map(cluster_assignments)

# Вывод нового DataFrame results_df
results_df

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
0,0,5160,0.003694,CGTGCCAGCAGCGCCCGGACTAGCGGGAGTAGGGACAATGAGCAGT...,RASSARTSGSRDNEQFF,TRBV7-3,TRBJ2-1,1
1,1,2317,0.001659,TGTGCCAGCAGTTACCGCGGGACAGCTTGGGAGACCCAGTACTTC,CASSYRGTAWETQYF,TRBV6-2,TRBJ2-5,2
2,2,2154,0.001542,TGTGCCAGCAGACCGCTTCTTGACAGAAACAATGAGCAGTTCTTC,CASRPLLDRNNEQFF,TRBV27,TRBJ2-1,3
3,3,2073,0.001484,TGCAGTGTCCTTTCTGGGGATTGGGGCATTCAGTACTTC,CSVLSGDWGIQYF,TRBV20-1,TRBJ2-4,4
4,4,1583,0.001133,TGTGCCAGCAGCCCGGGAGATCAGCCCCAGCATTTT,CASSPGDQPQHF,TRBV7-9,TRBJ1-5,5
...,...,...,...,...,...,...,...,...
9995,10956,7,0.000005,TGTGCCACCAGTGACCTGACAGAAGAGACCCAGTACTTC,CATSDLTEETQYF,TRBV24-1,TRBJ2-5,5936
9996,9786,7,0.000005,TGTGCCACCAGTGACGGACAGACCCACACAGATACGCAGTATTTT,CATSDGQTHTDTQYF,TRBV24-1,TRBJ2-3,8612
9997,10388,7,0.000005,TGTGCCACCAGTGACGGACAGGGTTACAATGAGCAGTTCTTC,CATSDGQGYNEQFF,TRBV24-1,TRBJ2-1,8613
9998,10389,7,0.000005,TGTGCCACCAGTGAGCTGGAACGGGCCGGGGAGCTGTTTTTT,CATSELERAGELFF,TRBV24-1,TRBJ2-2,8614


# Подсчёт Total Read Proportion

In [10]:
# Подсчет количества последовательностей в каждом кластере
cluster_counts = {i + 1: len(cluster) for i, cluster in enumerate(final_clusters)}
# Подсчет суммарного Read.proportion для каждого кластера
cluster_proportions = {}
for index, row in data_claster.iterrows():
  seq = row['CDR3.amino.acid.sequence']
  proportion = row['Read.proportion']
  for i, cluster in enumerate(final_clusters):
    if seq in cluster:
      cluster_proportions[i + 1] = cluster_proportions.get(i + 1, 0) + proportion
      break

# Создание DataFrame с суммарными пропорциями, номерами кластеров и количеством последовательностей
proportions_data = []
for cluster_number, total_proportion in cluster_proportions.items():
  proportions_data.append({'Cluster': cluster_number,
               'Total Read Proportion': total_proportion,
               'Sequence Count': cluster_counts[cluster_number]})


# Сортировка по Sequence Count

In [11]:
proportions_df = pd.DataFrame(proportions_data)
proportions_df = proportions_df.sort_values(by='Sequence Count', ascending=False)
proportions_df

,Cluster,Total Read Proportion,Sequence Count
76,78,0.001561,184
51,52,0.001672,110
329,331,0.000614,58
1173,56,0.000348,51
304,306,0.000314,31
...,...,...,...
3083,3091,0.000011,1
3082,3090,0.000011,1
3081,3089,0.000011,1
3080,3088,0.000011,1


# Сортировка по Total Read Proportion

In [12]:
proportions_df_sorted = proportions_df.sort_values(by='Total Read Proportion', ascending=False)
proportions_df_sorted = proportions_df_sorted.loc[proportions_df_sorted['Sequence Count'] > 1]
proportions_df_sorted

,Cluster,Total Read Proportion,Sequence Count
51,52,0.001672,110
76,78,0.001561,184
4,5,0.001163,4
329,331,0.000614,58
14,15,0.000528,4
...,...,...,...
8427,8434,0.000010,2
7926,7933,0.000010,2
7934,7941,0.000010,2
8282,8289,0.000010,2


# Выбор кластера

In [13]:
full = pd.merge(results_df, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [14]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,24,CASSLVLADNEQFF,0.000329,TRBV7-8,TRBJ2-1,TRBV27*01,TRBJ2-1*01,MHCI,HomoSapiens,CMV,pp65
1,52,CASSLGISYEQYF,0.000210,TRBV7-2,TRBJ2-7,TRBV27*01,TRBJ2-7*01,MHCI,HomoSapiens,CMV,pp65
2,56,CASSPSYEQYF,0.000191,TRBV7-2,TRBJ2-7,TRBV5-6*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BZLF1
3,58,CASSLGQTYEQYF,0.000190,TRBV7-2,TRBJ2-7,TRBV5-4*01,TRBJ2-7*01,MHCI,HomoSapiens,HomoSapiens,DNAJB1-PRKACA
4,78,CASSLTDTQYF,0.000163,TRBV7-9,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA4
...,...,...,...,...,...,...,...,...,...,...,...
694,306,CASSLVGNEQFF,0.000005,TRBV5-1,TRBJ2-1,TRBV5-1*01,TRBJ2-1*01,MHCI,HomoSapiens,SARS-CoV-2,Nucleocapsid
695,1620,CASSFRGDTEAFF,0.000005,TRBV5-1,TRBJ1-1,TRBV12-4*01,TRBJ1-1*01,MHCI,HomoSapiens,CMV,pp65
696,8531,CASSVTGGYGYTF,0.000005,TRBV5-1,TRBJ1-2,TRBV9*01,TRBJ1-2*01,MHCII,HomoSapiens,SARS-CoV-2,Spike
697,546,CASSRGTGELFF,0.000005,TRBV5-1,TRBJ2-2,TRBV12-3*01,TRBJ2-2*01,MHCI,HomoSapiens,CMV,pp65


In [15]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
7,79,CASSLEGQASSYEQYF,0.000161,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,LMP2A
8,79,CASSLEGQASSYEQYF,0.000161,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BMLF1
10,79,CASSLEGQASSYEQYF,0.000128,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,LMP2A
11,79,CASSLEGQASSYEQYF,0.000128,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCI,HomoSapiens,EBV,BMLF1
12,128,CASSDGGATDTQYF,0.000110,TRBV6-4,TRBJ2-3,TRBV6-4*01,TRBJ2-3*01,MHCI,HomoSapiens,HomoSapiens,MLANA
...,...,...,...,...,...,...,...,...,...,...,...
652,8312,CASSQALGSGANVLTF,0.000005,TRBV4-1,TRBJ2-6,TRBV4-1*01,TRBJ2-6*01,MHCI,HomoSapiens,CMV,pp65
675,52,CASSLDSSYEQYF,0.000005,TRBV5-1,TRBJ2-7,TRBV5-1*01,TRBJ2-7*01,MHCII,HomoSapiens,InfluenzaA,M1
677,331,CASSLENTGELFF,0.000005,TRBV5-1,TRBJ2-2,TRBV5-1*01,TRBJ2-2*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
684,78,CASSLGQETQYF,0.000005,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,CMV,IE1


In [16]:
full_vj['Cluster'].value_counts()

,count
Cluster,
78,11
3430,8
79,6
52,5
118,4
331,4
306,3
406,3
128,3


# Обработка для одного выбранного кластера

In [17]:
results_df_one_cluster = results_df.loc[results_df['Cluster'] == 78]
results_df_one_cluster = results_df_one_cluster.rename(columns={'Sequence': 'CDR3.amino.acid.sequence'})
results_df_one_cluster

,Rank,Read.count,Read.proportion,CDR3.nucleotide.sequence,CDR3.amino.acid.sequence,bestVGene,bestJGene,Cluster
77,77,227,0.000163,TGTGCCAGCAGCTTGACTGATACGCAGTATTTT,CASSLTDTQYF,TRBV7-9,TRBJ2-3,78
622,628,53,0.000038,TGTGCCAGCAGCTTAGGAGATACGCAGTATTTT,CASSLGDTQYF,TRBV7-2,TRBJ2-3,78
899,911,41,0.000029,TGTGCCAGCAGCTACACAGATACGCAGTATTTT,CASSYTDTQYF,TRBV7-2,TRBJ2-3,78
930,942,40,0.000029,TGTGCCAGCAGCTACCGCACAGATACGCAGTATTTT,CASSYRTDTQYF,TRBV3-2,TRBJ2-3,78
958,969,39,0.000028,TGTGCCAGCAGCCGTGGAGATACGCAGTATTTT,CASSRGDTQYF,TRBV3-1,TRBJ2-3,78
...,...,...,...,...,...,...,...,...
9824,10763,7,0.000005,TGCGCCAGCAGCTTGGCCGGTTTAAAAGAGACCCAGTACTTC,CASSLAGLKETQYF,TRBV5-1,TRBJ2-5,78
9838,11517,7,0.000005,TGCGCCAGCAGCTTGGGGCAAGAGACCCAGTACTTC,CASSLGQETQYF,TRBV5-1,TRBJ2-5,78
9839,11247,7,0.000005,TGCGCCAGCAGCTTGGGGCAGCAAGAGACCCAGTACTTC,CASSLGQQETQYF,TRBV5-1,TRBJ2-5,78
9877,11241,7,0.000005,TGCGCCAGCAGCTTTCCTAGCACAGATACGCAGTATTTT,CASSFPSTDTQYF,TRBV5-1,TRBJ2-3,78


In [18]:
full = pd.merge(results_df_one_cluster, database_exp , on='CDR3.amino.acid.sequence', how='inner')

In [19]:
full = full[['Cluster', 'CDR3.amino.acid.sequence', 'Read.proportion', 'bestVGene',	'bestJGene', 'v.segm',	'j.segm', 'mhc.class', 'species', 'antigen.species', 'antigen.gene']]
full

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
0,78,CASSLTDTQYF,0.000163,TRBV7-9,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA4
1,78,CASSLTDTQYF,0.000163,TRBV7-9,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,BZLF1
2,78,CASSLTDTQYF,0.000163,TRBV7-9,TRBJ2-3,TRBV7-2*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,pp50
3,78,CASSLGDTQYF,0.000038,TRBV7-2,TRBJ2-3,TRBV12-1*01,TRBJ2-5*01,MHCI,MusMusculus,MCMV,M38
4,78,CASSFTDTQYF,0.000025,TRBV12-4,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,pp65
...,...,...,...,...,...,...,...,...,...,...,...
160,78,CASSLSTDTQYF,0.000005,TRBV5-1,TRBJ2-3,TRBV12-3*01,TRBJ2-3*01,MHCI,HomoSapiens,SARS-CoV-2,ORF1ab
161,78,CASSLSTDTQYF,0.000005,TRBV5-1,TRBJ2-3,TRBV28*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,pp65
162,78,CASSLSTDTQYF,0.000005,TRBV5-1,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1
163,78,CASSLSTDTQYF,0.000005,TRBV5-1,TRBJ2-3,TRBV5-4*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA3A


# Разборки с V и J

In [20]:
full_vj = full.loc[(full['bestVGene'].str[:7] == full['v.segm'].str[:7]) & (full['bestJGene'].str[:7] == full['j.segm'].str[:7])]
full_vj

,Cluster,CDR3.amino.acid.sequence,Read.proportion,bestVGene,bestJGene,v.segm,j.segm,mhc.class,species,antigen.species,antigen.gene
5,78,CASSFTDTQYF,0.000025,TRBV12-4,TRBJ2-3,TRBV12-4*01,TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1
46,78,CASSLGETQYF,0.000011,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
49,78,CASSLGETQYF,0.000011,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,HomoSapiens,IGF2BP2
91,78,CASSLGETQYF,0.000008,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCII,HomoSapiens,CMV,pp65
94,78,CASSLGETQYF,0.000008,TRBV5-1,TRBJ2-5,TRBV5-1*01,TRBJ2-5*01,MHCI,HomoSapiens,HomoSapiens,IGF2BP2
132,78,CASSLDTQYF,0.000006,TRBV7-9,TRBJ2-3,TRBV7-9*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,BMLF1
133,78,CASSLDTQYF,0.000006,TRBV7-9,TRBJ2-3,TRBV7-9*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,BZLF1
134,78,CASSLDTQYF,0.000006,TRBV7-9,TRBJ2-3,TRBV7-9*01,TRBJ2-3*01,MHCI,HomoSapiens,InfluenzaA,M
135,78,CASSLDTQYF,0.000006,TRBV7-9,TRBJ2-3,TRBV7-9*01,TRBJ2-3*01,MHCI,HomoSapiens,EBV,EBNA4
136,78,CASSLDTQYF,0.000006,TRBV7-9,TRBJ2-3,"TRBV7-9*01,TRBV11-2*01",TRBJ2-3*01,MHCI,HomoSapiens,CMV,IE1


#Read Propotion

In [ ]:
from matplotlib import pyplot as plt
full_vj['Read.proportion'].plot(kind='line', figsize=(8, 4), title='Read.proportion')
plt.gca().spines[['top', 'right']].set_visible(False)